In [1]:
# boilerplate sanity check
import os,warnings,cpuinfo,multiprocessing,psutil,sys
warnings.simplefilter(action='ignore',category=FutureWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
try:cpu=cpuinfo.get_cpu_info();print('CPU says hello!\n-name: {}\n-architecture: {}\n-bits: {}\n-frequency: {}\n-threads: {}\n-l3_cache: {}\n-l2_cache: {}\n-l1_cache: {}'.format(cpu['brand'],cpu['arch'],cpu['bits'],cpu['hz_actual'],multiprocessing.cpu_count(),cpu['l3_cache_size'],cpu['l2_cache_size'],cpu['l1_data_cache_size']))
except:print('Failed to get all CPU info!')
try:print('RAM says hello!\n-virtual: {} GB\n-swap: {} GB'.format(round(psutil.virtual_memory().total/1024**3,2),round(psutil.swap_memory().total/1024**3,2)))
except:print('Failed to get all RAM info!')
print('Python says hello!\n-python: {}'.format(sys.version.split(' ')[0]))
try:
    import tensorflow as tf;import keras;from tensorflow.python.client import device_lib;from tensorflow.keras import mixed_precision;from tensorflow.python.framework.ops import disable_eager_execution;gpus=[];
    print('-tensorflow: {}'.format(tf.version.VERSION))
    print('-keras: {}'.format(keras.__version__))
    for device in device_lib.list_local_devices():
        try:
            if device.device_type=='GPU':
                name=device.name
                try:
                    splitted=device.physical_device_desc.split(',')
                    for part in splitted:
                        if'name:'in part or'Name:'in part:name=part.split(':')[1].strip()
                except:pass
                gpus.append('\n-name: {}\n-vram: {} GB'.format(name,round(device.memory_limit/1000**3,2)))
        except:pass
    if len(gpus)>0:
        print('GPU{} hello!{}'.format('s saying'if len(gpus)>1 else' says',''.join(gpus)))
        try:
            smi=os.popen('nvidia-smi').read().split('\n')[2];d=smi.find('Driver Version')+16;c=smi.find('CUDA Version')+14
            print('Drivers saying hello!\n-nvidia: {}\n-cuda: {}'.format(smi[d:d+10].strip(),smi[c:c+5].strip()))
        except:pass
    else:print('GPU not found!')
except:print('GPU not found! Check tensorflow-gpu and cudatoolkit installations!')
# print('===============================')
# try:
#     disable_eager_execution()
#     print('Eager execution disabled!')
# except:
#     print('Failed to disable eager execution!')
# try:
#     mixed_precision.set_global_policy('mixed_float16')
#     print('Mixed precision enabled!')
# except:
#     print('Failed to enable mixed precision!')

CPU says hello!
-name: 12th Gen Intel(R) Core(TM) i5-12600KF
-architecture: X86_64
-bits: 64
-frequency: 3.6865 GHz
-threads: 16
-l3_cache: 16384 KB
-l2_cache: 64 MiB
-l1_cache: 512 KiB
RAM says hello!
-virtual: 102.17 GB
-swap: 256.0 GB
Python says hello!
-python: 3.9.19


2024-10-23 14:28:12.739184: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-23 14:28:12.753533: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-23 14:28:12.757203: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


-tensorflow: 2.17.0
-keras: 3.5.0
GPU says hello!
-name: Tesla P40
-vram: 23.34 GB
Drivers saying hello!
-nvidia: 560.35.03
-cuda: 12.6


In [2]:
import os, warnings
warnings.simplefilter(action='ignore',category=FutureWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

import sys
sys.path.insert(1,'source')
from DataGenerator import DataGenerator
import numpy as np

In [4]:
datagen_props = {
    'path'          : 'source/data',     #path of the data
    'seed'          : 42,         #seed for the split
    'split'         : 0.8,        #train/all ratio
    'train'         : True,       #training/testing split
    'control'       : True,       #include control data points
    'huntington'    : False,      #include huntington data points
    'batch_size'    : 50000,      #batch size
    'spatial'       : False,      #keep spaital format of flatten voxels in the brain region
    'left'          : True,       #include left hemisphere data (if both false, concatenate the left and right hemisphere layers)
    'right'         : True,       #include right hemisphere data
    'threshold'     : 0.5,        #if float value provided, it thresholds the connectivty map
    'binarize'      : False,      #only works if threshold if greater or equal than half, and then it binarizes the connectivity map
    'not_connected' : False,      #only works if thresholded and not single, and then it appends an extra encoding for the 'not connected'
    'single'        : 0,          #if int index value is provided, it only returns a specified connectivity map
    'target'        : False,
    'roi'           : False,
    'brain'         : False,
    'features'      : [],         #used radiomics features (emptylist means all)
    'features_vox'  : [],         #used voxel based radiomics features (emptylist means all)
    'radiomics'     : ['b25'],    #used radiomics features bin settings
    'radiomics_vox' : ['k5_b25'], #used voxel based radiomics features kernel and bin settings
}

In [5]:
datagen_props1 = datagen_props.copy()
train = DataGenerator(**datagen_props1)
datagen_props2 = datagen_props.copy()
datagen_props2['train'] = False
test = DataGenerator(**datagen_props2)

print(len(train))

14


In [5]:
model = Sequential(name='FFN')
model.add(Input(train.x_shape))
model.add(Dense(128,activation='silu'))
model.add(Dense(128,activation='silu'))
model.add(Dense(128,activation='silu'))
model.add(Dense(128,activation='silu'))
model.add(Dense(train.y_shape[0],activation='silu'))
#model.summary()

model.compile(loss='mean_absolute_error', optimizer='adam')
history = model.fit(
    train,
    steps_per_epoch=len(train),
    validation_data=test,
    validation_steps=len(test),
    epochs=10,
    verbose=1,
)

/projects/anaconda3/envs/tensorflow217python39/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


2024-10-23 14:20:52.926921: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
/projects/anaconda3/envs/tensorflow217python39/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


I0000 00:00:1729693360.979254     326 service.cc:146] XLA service 0x7691f800cef0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729693360.979352     326 service.cc:154]   StreamExecutor device (0): Tesla P40, Compute Capability 6.1


 1/14 ━━━━━━━━━━━━━━━━━━━━ 24:39 114s/step - loss: 0.0557

...

  File "/projects/anaconda3/envs/tensorflow217python39/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()


KeyboardInterrupt


KeyboardInterrupt


KeyboardInterrupt


KeyboardInterrupt
